In [6]:
import pandas as pd
import PIL
import PIL.Image
import tensorflow as tf

In [3]:
data = pd.read_pickle('processed_img.pkl')

In [8]:
data.head()

,site_id,patient_id,image_id,laterality,view,age,cancer,biopsy,invasive,BIRADS,implant,density,machine_id,difficult_negative_case,image_path,img_array
0,2,10006,462822612,L,CC,61.0,0,0,0,NaN,0,NaN,29,False,train_images/10006/462822612.dcm,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
1,2,10006,1459541791,L,MLO,61.0,0,0,0,NaN,0,NaN,29,False,train_images/10006/1459541791.dcm,"[[199, 195, 218, 218, 207, 218, 206, 199, 199,..."
2,2,10006,1864590858,R,MLO,61.0,0,0,0,NaN,0,NaN,29,False,train_images/10006/1864590858.dcm,"[[180, 192, 197, 198, 196, 196, 204, 197, 195,..."
3,2,10006,1874946579,R,CC,61.0,0,0,0,NaN,0,NaN,29,False,train_images/10006/1874946579.dcm,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
4,2,10011,220375232,L,CC,55.0,0,0,0,0.0,0,NaN,21,True,train_images/10011/220375232.dcm,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."


In [13]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator()

cancer_map = {0: 'negative', 1: 'positive'}

data['label'] = data['cancer'].map(cancer_map)

data.head()

,site_id,patient_id,image_id,laterality,view,age,cancer,biopsy,invasive,BIRADS,implant,density,machine_id,difficult_negative_case,image_path,img_array,label
0,2,10006,462822612,L,CC,61.0,0,0,0,NaN,0,NaN,29,False,train_images/10006/462822612.dcm,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",negative
1,2,10006,1459541791,L,MLO,61.0,0,0,0,NaN,0,NaN,29,False,train_images/10006/1459541791.dcm,"[[199, 195, 218, 218, 207, 218, 206, 199, 199,...",negative
2,2,10006,1864590858,R,MLO,61.0,0,0,0,NaN,0,NaN,29,False,train_images/10006/1864590858.dcm,"[[180, 192, 197, 198, 196, 196, 204, 197, 195,...",negative
3,2,10006,1874946579,R,CC,61.0,0,0,0,NaN,0,NaN,29,False,train_images/10006/1874946579.dcm,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",negative
4,2,10011,220375232,L,CC,55.0,0,0,0,0.0,0,NaN,21,True,train_images/10011/220375232.dcm,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",negative


In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from tqdm import tqdm
import dicomsdl as dicom
import PIL
from PIL import Image as im
import cv2

In [30]:
for row in tqdm(range(len(data['img_array']))):
    if data['cancer'][row] == 0:
        im.fromarray(negative_cases['img_array'][row]).save('converted_images/negative/pic_test_' + str(row) + '.png')
    if data['cancer'][row] == 1:
        im.fromarray(positive_cases['img_array'][row]).save('converted_images/positive/pic_test_' + str(row) + '.png')

100%|██████████| 54706/54706 [02:30<00:00, 364.31it/s]


In [40]:
train_tf = tf.keras.utils.image_dataset_from_directory(
    'converted_images',
    labels="inferred",
    label_mode="binary",
    class_names=None,
    color_mode="grayscale",
    batch_size=32,
    image_size=(256, 256),
    shuffle=True,
    seed=42,
    validation_split=0.3,
    subset='training',
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False
)

Found 54706 files belonging to 2 classes.
Using 38295 files for training.


In [41]:
val_tf = tf.keras.utils.image_dataset_from_directory(
    'converted_images',
    labels="inferred",
    label_mode="binary",
    class_names=None,
    color_mode="grayscale",
    batch_size=32,
    image_size=(256, 256),
    shuffle=True,
    seed=42,
    validation_split=0.3,
    subset='validation',
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False
)

Found 54706 files belonging to 2 classes.
Using 16411 files for validation.


In [36]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [37]:
# Create model
model = keras.models.Sequential()

# Adding model layers
model.add(keras.layers.Conv2D(32, (5, 5), activation='relu', input_shape=(256, 256, 1)))
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(keras.layers.Conv2D(64, (5, 5), activation='relu'))
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(keras.layers.Conv2D(128, (5, 5), activation='relu'))
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(128, activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(2, activation='softmax'))


In [38]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

In [39]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 252, 252, 32)      832       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 126, 126, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 122, 122, 64)      51264     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 61, 61, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 57, 57, 128)       204928    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 28, 28, 128)       0         
_________________________________________________________________
flatten (Flatten)            (None, 100352)            0

In [47]:
itr_train = make_iterator(train_tf)
itr_valid = make_iterator(val_tf)

NameError: name 'make_iterator' is not defined

In [46]:
model_fit = model.fit_generator(
        generator = train_tf,
        validation_data = val_tf,
        validation_steps=32,
        epochs=10,
        verbose=1,)

Epoch 1/10


c:\Users\marcj\anaconda3\envs\Tensorflow_GPU\lib\site-packages\keras\engine\training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


ValueError: in user code:

    c:\Users\marcj\anaconda3\envs\Tensorflow_GPU\lib\site-packages\keras\engine\training.py:853 train_function  *
        return step_function(self, iterator)
    c:\Users\marcj\anaconda3\envs\Tensorflow_GPU\lib\site-packages\keras\engine\training.py:842 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    c:\Users\marcj\anaconda3\envs\Tensorflow_GPU\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    c:\Users\marcj\anaconda3\envs\Tensorflow_GPU\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    c:\Users\marcj\anaconda3\envs\Tensorflow_GPU\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, **kwargs)
    c:\Users\marcj\anaconda3\envs\Tensorflow